In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
spark = SparkSession.builder.master('local').appName('aggDataFrames').enableHiveSupport().getOrCreate()
sc=spark.sparkContext

In [6]:
orders = spark.read.csv('data//retail_db//orders.csv',header=True,inferSchema=True)
order_items = spark.read.csv('data//retail_db//order_items.csv',header=True,inferSchema=True)
customers = spark.read.csv('data//retail_db//customers.csv',header=True,inferSchema=True)
products = spark.read.csv('data//retail_db//products.csv',header=True,inferSchema=True)
categories = spark.read.csv('data//retail_db//categories.csv',header=True,inferSchema=True)
departments = spark.read.csv('data//retail_db//departments.csv',header=True,inferSchema=True)

# 1. Top N orders for the day based on order revenue

In [7]:
p11=orders.join(order_items, orders.order_id == order_items.order_item_order_id).groupBy(orders.order_date,orders.order_id). \
    agg(round(sum(order_items.order_item_subtotal),2).alias('order_revenue'))

In [8]:
p12=p11.withColumn('rnk',rank().over(Window.partitionBy(orders.order_date).orderBy(col('order_revenue').desc())))

In [9]:
p1=p12.filter(col('rnk') <= 5).drop('rnk').orderBy('order_date',col('order_revenue').desc())

In [10]:
p1.show()

+-------------------+--------+-------------+
|         order_date|order_id|order_revenue|
+-------------------+--------+-------------+
|2013-07-25 00:00:00|   57779|       1649.8|
|2013-07-25 00:00:00|      84|      1499.87|
|2013-07-25 00:00:00|      12|      1299.87|
|2013-07-25 00:00:00|      73|      1279.65|
|2013-07-25 00:00:00|      28|       1159.9|
|2013-07-26 00:00:00|     164|      1369.81|
|2013-07-26 00:00:00|     284|      1349.83|
|2013-07-26 00:00:00|     165|      1333.84|
|2013-07-26 00:00:00|     188|      1299.91|
|2013-07-26 00:00:00|     292|      1299.86|
|2013-07-27 00:00:00|     417|      1379.88|
|2013-07-27 00:00:00|   57812|      1259.93|
|2013-07-27 00:00:00|     399|      1259.87|
|2013-07-27 00:00:00|     477|      1249.91|
|2013-07-27 00:00:00|     442|      1229.91|
|2013-07-28 00:00:00|     614|      1499.87|
|2013-07-28 00:00:00|   57850|      1259.94|
|2013-07-28 00:00:00|     517|      1209.91|
|2013-07-28 00:00:00|   57861|      1209.77|
|2013-07-2

### SQL

In [ ]:
spark.sql(" SELECT order_date, order_id, order_revenue FROM \
(SELECT o.order_date, o.order_id, round(sum(oi.order_item_subtotal),2) order_revenue, \
RANK() OVER (PARTITION BY o.order_date ORDER BY round(sum(oi.order_item_subtotal),2) DESC) rnk  \
FROM orders o, order_items oi WHERE o.order_id = oi.order_item_order_id GROUP BY  o.order_date, o.order_id ) main WHERE rnk<= 5 ORDER BY 1, 3 DESC").show()


# 2. Get average revenue per day and all the orders which are more than average.

In [12]:
p21=orders.join(order_items, orders.order_id == order_items.order_item_order_id).groupBy('order_date','order_id').\
agg(round(sum('order_item_subtotal'),2).alias('order_revenue'))

In [13]:
p22=p21.withColumn('avg_revenue',avg('order_revenue').over(Window.partitionBy('order_date')))

In [14]:
p2=p22.filter('order_revenue >= avg_revenue').orderBy('order_date',col('order_revenue').desc())

In [15]:
p2.show()

+-------------------+--------+-------------+-----------------+
|         order_date|order_id|order_revenue|      avg_revenue|
+-------------------+--------+-------------+-----------------+
|2013-07-25 00:00:00|   57779|       1649.8|587.5330172413794|
|2013-07-25 00:00:00|      84|      1499.87|587.5330172413794|
|2013-07-25 00:00:00|      12|      1299.87|587.5330172413794|
|2013-07-25 00:00:00|      73|      1279.65|587.5330172413794|
|2013-07-25 00:00:00|      28|       1159.9|587.5330172413794|
|2013-07-25 00:00:00|      62|      1149.94|587.5330172413794|
|2013-07-25 00:00:00|   57764|      1149.92|587.5330172413794|
|2013-07-25 00:00:00|       5|      1129.86|587.5330172413794|
|2013-07-25 00:00:00|   57788|      1119.86|587.5330172413794|
|2013-07-25 00:00:00|      29|      1109.85|587.5330172413794|
|2013-07-25 00:00:00|      99|      1099.93|587.5330172413794|
|2013-07-25 00:00:00|   57757|      1099.87|587.5330172413794|
|2013-07-25 00:00:00|   57782|      1049.85|587.5330172

### SQL

In [ ]:
spark.sql(" \
SELECT order_date, order_id, order_revenue, avg_revenue FROM \
(SELECT o.order_date, o.order_id, sum(oi.order_item_subtotal) order_revenue, \
AVG(sum(oi.order_item_subtotal)) OVER (PARTITION BY o.order_date ) avg_revenue  \
FROM orders o, order_items oi WHERE o.order_id = oi.order_item_order_id GROUP BY  o.order_date, o.order_id) \
WHERE order_revenue >= avg_revenue ORDER BY 1, 3 DESC").show()

# 3. Top N products for the day

In [16]:
p31=orders.join(order_items, orders.order_id == order_items.order_item_order_id).join(products, order_items.order_item_product_id == products.product_id).groupBy('order_date','product_name')

In [17]:
p32=p31.agg(round(sum('order_item_subtotal'),2).alias('product_revenue')).withColumn('rnk',rank().over(Window.partitionBy('order_date').orderBy(col('product_revenue').desc())))

In [18]:
p33=p32.filter(col('rnk') <=5).orderBy('order_date',col('product_revenue').desc())

In [19]:
p3=p33

In [20]:
p3.show()

+-------------------+--------------------+---------------+---+
|         order_date|        product_name|product_revenue|rnk|
+-------------------+--------------------+---------------+---+
|2013-07-25 00:00:00|Field & Stream Sp...|       10799.46|  1|
|2013-07-25 00:00:00|Diamondback Women...|        9599.36|  2|
|2013-07-25 00:00:00|Nike Men's Free 5...|        8499.15|  3|
|2013-07-25 00:00:00|Perfect Fitness P...|        7558.74|  4|
|2013-07-25 00:00:00|Pelican Sunstream...|        6999.65|  5|
|2013-07-26 00:00:00|Field & Stream Sp...|       28798.56|  1|
|2013-07-26 00:00:00|Perfect Fitness P...|        20996.5|  2|
|2013-07-26 00:00:00|Diamondback Women...|       16798.88|  3|
|2013-07-26 00:00:00|Nike Men's Free 5...|        14998.5|  4|
|2013-07-26 00:00:00|Pelican Sunstream...|       10799.46|  5|
|2013-07-27 00:00:00|Field & Stream Sp...|        21998.9|  1|
|2013-07-27 00:00:00|Diamondback Women...|       14399.04|  2|
|2013-07-27 00:00:00|Nike Men's Free 5...|       10698.

### SQL

In [ ]:
spark.sql('SELECT * FROM \
( SELECT o.order_date, p.product_name, round(sum(oi.order_item_subtotal),2) product_revenue, \
RANK() OVER(PARTITION BY o.order_date ORDER BY round(sum(oi.order_item_subtotal),2) DESC) rnk \
from orders o, order_items oi, products p WHERE o.order_id = oi.order_item_order_id AND oi.order_item_product_id = p.product_id \
GROUP BY o.order_date, p.product_name ) WHERE rnk <=5 ORDER BY 1,3 DESC').show()


# 4. Percentage of order items in Order revenue


In [21]:
p41=order_items.withColumn('order_revenue',sum('order_item_subtotal').over(Window.partitionBy('order_item_order_id'))).\
withColumn('pcnt', round(col('order_item_subtotal')/col('order_revenue'),2))

In [22]:
p42=p41.select('order_item_order_id','order_item_id','order_item_subtotal','order_revenue','pcnt').orderBy(col('order_item_order_id'))

In [23]:
p4=p42

In [24]:
p4.show()

+-------------------+-------------+-------------------+------------------+----+
|order_item_order_id|order_item_id|order_item_subtotal|     order_revenue|pcnt|
+-------------------+-------------+-------------------+------------------+----+
|                  1|            1|             299.98|            299.98| 1.0|
|                  2|            3|              250.0|            579.98|0.43|
|                  2|            4|             129.99|            579.98|0.22|
|                  2|            2|             199.99|            579.98|0.34|
|                  4|            8|             199.92|            699.85|0.29|
|                  4|            7|              150.0|            699.85|0.21|
|                  4|            6|             299.95|            699.85|0.43|
|                  4|            5|              49.98|            699.85|0.07|
|                  5|           10|             299.95|1129.8600000000001|0.27|
|                  5|           12|     

### SQL

In [ ]:
spark.sql("SELECT order_item_order_id, order_item_id, order_item_subtotal, \
round(SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2) order_revenue , \
(round(order_item_subtotal /SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2)) pcnt FROM order_items ORDER BY 1 ").show()



# 5. Difference in top 2 order items for the order revenue

In [25]:
p51= order_items.withColumn('rnk',rank().over(Window.partitionBy('order_item_order_id').orderBy(col('order_item_subtotal').desc()))).withColumn('next',lead('order_item_subtotal').over(Window.partitionBy('order_item_order_id').orderBy(col('order_item_subtotal').desc())))

In [26]:
p52=p51.select('order_item_order_id','order_item_id','order_item_subtotal','next','rnk')

In [27]:
p53=p52.orderBy('order_item_order_id','rnk').filter(col('rnk')==1).fillna(0).withColumn('diff',round(col('order_item_subtotal').cast('double')-col('next').cast('double'),2)).drop('rnk')

In [28]:
p5=p53

In [29]:
p5.show()

+-------------------+-------------+-------------------+------+------+
|order_item_order_id|order_item_id|order_item_subtotal|  next|  diff|
+-------------------+-------------+-------------------+------+------+
|                  1|            1|             299.98|   0.0|299.98|
|                  2|            3|              250.0|199.99| 50.01|
|                  4|            6|             299.95|199.92|100.03|
|                  5|           12|             299.98|299.95|  0.03|
|                  5|            9|             299.98|299.98|   0.0|
|                  7|           15|             299.98|199.99| 99.99|
|                  8|           18|             299.95|199.92|100.03|
|                  9|           23|             199.99|199.98|  0.01|
|                  9|           22|             199.99|199.99|   0.0|
|                 10|           24|             199.99|199.99|   0.0|
|                 10|           28|             199.99|129.99|  70.0|
|                 11

### SQL

In [ ]:
spark.sql("SELECT s.*, s.order_item_subtotal - s.next_item diff FROM ( SELECT order_item_order_id, order_item_id, order_item_subtotal , LEAD(order_item_subtotal) OVER(PARTITION BY order_item_order_id ORDER BY order_item_subtotal DESC) next_item, RANK() OVER(PARTITION BY order_item_order_id ORDER BY order_item_subtotal DESC) rnk FROM order_items) s WHERE rnk =  1 ORDER BY 1, 5 ").show()


# 6. Get order items contributing more than 75% of the total order

In [30]:
p61 = order_items.withColumn('order_revenue',sum('order_item_subtotal').over(Window.partitionBy('order_item_order_id'))).\
withColumn('pcnt',round(col('order_item_subtotal')/col('order_revenue'),2))

In [31]:
p62= p61.filter(col('pcnt') >= 0.75).drop('order_item_product_id','order_item_quantity','order_item_product_price').orderBy('order_item_order_id')

In [32]:
p6=p62

In [34]:
p6.show()

+-------------+-------------------+-------------------+-------------+----+
|order_item_id|order_item_order_id|order_item_subtotal|order_revenue|pcnt|
+-------------+-------------------+-------------------+-------------+----+
|            1|                  1|             299.98|       299.98| 1.0|
|           39|                 13|             127.96|       127.96| 1.0|
|           68|                 23|             299.98|       299.98| 1.0|
|           74|                 25|             399.98|       399.98| 1.0|
|           88|                 30|              100.0|        100.0| 1.0|
|           89|                 31|             499.95|       499.95| 1.0|
|           94|                 34|             299.98|       299.98| 1.0|
|           95|                 35|             129.99|       129.99| 1.0|
|          103|                 39|             199.99|       199.99| 1.0|
|          112|                 44|             399.98|       399.98| 1.0|
|          127|          

### SQL


In [ ]:
spark.sql("SELECT s.*  FROM (SELECT order_item_id, order_item_order_id, order_item_subtotal, round(sum(order_item_subtotal) over(partition by order_item_order_id),2) order_revenue , round(order_item_subtotal/round(sum(order_item_subtotal) over(partition by order_item_order_id),2),2) pcnt FROM order_items) s WHERE pcnt >= 0.75 ORDER BY 1").show()


# 7.What are the best-selling and the second best-selling products in every category?

In [35]:
p71 = order_items.join(products, order_items.order_item_product_id == products.product_id).join(categories, categories.category_id == products.product_category_id).groupBy('category_name','product_name').agg(round(sum(order_items.order_item_subtotal),2).alias('product_revenue'))

In [36]:
p72=p71.withColumn('rnk',rank().over(Window.partitionBy('category_name').orderBy(col('product_revenue').desc()))).filter(col('rnk') <=2).orderBy('category_name','rnk')

In [37]:
p7=p72

In [39]:
p7.show()

+-------------------+--------------------+---------------+---+
|      category_name|        product_name|product_revenue|rnk|
+-------------------+--------------------+---------------+---+
|        Accessories|Team Golf St. Lou...|       23940.42|  1|
|        Accessories|Team Golf Tenness...|       22865.85|  2|
|    As Seen on  TV!|Nike Men's Free T...|       20597.94|  1|
|Baseball & Softball|adidas Men's F10 ...|       56330.61|  1|
|Baseball & Softball|adidas Kids' F5 M...|       27327.19|  2|
|         Basketball| SOLE E25 Elliptical|         9999.9|  1|
|         Basketball|Diamondback Boys'...|        8699.71|  2|
|       Boxing & MMA|Under Armour Wome...|       46559.59|  1|
|       Boxing & MMA|Nike Women's Free...|       19567.94|  2|
|   Camping & Hiking|Diamondback Women...|     4118425.42|  1|
|   Cardio Equipment|Nike Men's Free 5...|      3667633.2|  1|
|   Cardio Equipment|Nike Women's Temp...|        27210.0|  2|
|             Cleats|Perfect Fitness P...|     4421143.

### SQL

In [ ]:
spark.sql("SELECT * FROM (SELECT c.category_name, p.product_name, round(sum(oi.order_item_subtotal),2) product_revenue, rank() over(partition by c.category_name ORDER BY round(sum(oi.order_item_subtotal)) DESC) rnk FROM order_items oi, products p, categories c WHERE oi.order_item_product_id = p.product_id AND p.product_category_id = c.category_id GROUP BY c.category_name, p.product_name ) WHERE rnk<= 2 ORDER BY 1,4").show()

# 8. What is the difference between the revenue of each product and the revenue of the best-selling product in the same category of that product?

In [42]:
p8=order_items.join(products, products.product_id == order_items.order_item_product_id).\
join(categories, categories.category_id ==  products.product_category_id).\
groupBy('category_name','product_name').agg(round(sum('order_item_subtotal'),2).alias('product_revenue')).\
withColumn('best',max(col('product_revenue')).over(Window.partitionBy('category_name'))).withColumn('diff_prices',col('best')-col('product_revenue')).\
orderBy('category_name','product_revenue')

In [43]:
p8.show()

+-------------------+--------------------+---------------+----------+------------------+
|      category_name|        product_name|product_revenue|      best|       diff_prices|
+-------------------+--------------------+---------------+----------+------------------+
|        Accessories|Team Golf New Eng...|       20566.77|  23940.42| 3373.649999999998|
|        Accessories|Team Golf San Fra...|       21766.29|  23940.42|2174.1299999999974|
|        Accessories|Team Golf Pittsbu...|       22166.13|  23940.42|1774.2899999999972|
|        Accessories|Team Golf Texas L...|       22366.05|  23940.42| 1574.369999999999|
|        Accessories|Team Golf Tenness...|       22865.85|  23940.42|1074.5699999999997|
|        Accessories|Team Golf St. Lou...|       23940.42|  23940.42|               0.0|
|    As Seen on  TV!|Nike Men's Free T...|       20597.94|  20597.94|               0.0|
|Baseball & Softball|adidas Brazuca 20...|       10399.35|  56330.61|          45931.26|
|Baseball & Softball|

### SQL

In [ ]:
spark.sql("SELECT s.*, best_product - product_revenue diff_prices FROM (SELECT c.category_name, p.product_name, ROUND(SUM(oi.order_item_subtotal),2) product_revenue, MAX(ROUND(SUM(oi.order_item_subtotal),2)) OVER(PARTITION BY c.category_name) best_product FROM categories c, products p , order_items oi WHERE oi.order_item_product_id = p.product_id AND p.product_category_id = c.category_id GROUP BY c.category_name, p.product_name) s ORDER BY 1,3").show()



# 9.Most selling product (But Quantity not by Cost) for every month in the database (Between July 2013 to July 2014) 

In [44]:
p91=orders.join(order_items, orders.order_id == order_items.order_item_order_id).join(products, products.product_id == order_items.order_item_product_id).withColumn('date_month',date_format(orders.order_date,'yyyyMM').cast('bigint')).filter(col('date_month').between(201307,201407))

In [45]:
p92=p91.groupBy(col('date_month'),'product_name').agg(sum(order_items.order_item_quantity).alias('order_quantity'))

In [46]:
p93=p92.withColumn('rnk',rank().over(Window.partitionBy(col('date_month')).orderBy(col('order_quantity').desc()))).filter(col('rnk')==1).orderBy(col('date_month'))

In [48]:
p9=p93
p9.show()

+----------+--------------------+--------------+---+
|date_month|        product_name|order_quantity|rnk|
+----------+--------------------+--------------+---+
|    201307|Perfect Fitness P...|          1710|  1|
|    201308|Perfect Fitness P...|          6089|  1|
|    201309|Perfect Fitness P...|          6153|  1|
|    201310|Perfect Fitness P...|          5699|  1|
|    201311|Perfect Fitness P...|          6745|  1|
|    201312|Perfect Fitness P...|          5983|  1|
|    201401|Perfect Fitness P...|          6340|  1|
|    201402|Perfect Fitness P...|          6315|  1|
|    201403|Perfect Fitness P...|          6115|  1|
|    201404|Perfect Fitness P...|          5959|  1|
|    201405|Perfect Fitness P...|          6014|  1|
|    201406|Perfect Fitness P...|          5873|  1|
|    201407|Perfect Fitness P...|          4703|  1|
+----------+--------------------+--------------+---+



### SQL

In [ ]:
spark.sql("SELECT * FROM (SELECT date_format(o.order_date,'yyyyMM') order_month, p.product_name, sum(oi.order_item_quantity) order_quantity, rank() over(partition by date_format(o.order_date,'yyyyMM') order by sum(oi.order_item_quantity) desc) rnk from orders o, order_items oi, products p WHERE oi.order_item_order_id = o.order_id and oi.order_item_product_id = p.product_id GROUP BY date_format(o.order_date,'yyyyMM'), p.product_name) s WHERE rnk=1 ORDER BY 1").show()


# 10. Who are the top 10 revenue generating customers

In [49]:
orders.join(order_items, orders.order_id ==  order_items.order_item_order_id).join(customers, customers.customer_id ==  orders.order_customer_id).\
groupBy(customers.customer_id, concat_ws(' ',customers.customer_fname,customers.customer_lname)).\
agg(sum(order_items.order_item_subtotal).alias('customer_revenue')).orderBy(col('customer_revenue').desc()).limit(10).show()


+-----------+--------------------------------------------+------------------+
|customer_id|concat_ws( , customer_fname, customer_lname)|  customer_revenue|
+-----------+--------------------------------------------+------------------+
|        791|                                  Mary Smith|10524.169999999993|
|       9371|                              Mary Patterson| 9299.029999999997|
|       8766|                                 Mary Duncan| 9296.139999999998|
|       1657|                              Betty Phillips| 9223.709999999994|
|       2641|                                Betty Spears| 9130.919999999995|
|       1288|                             Evelyn Thompson| 9019.109999999995|
|       3710|                                Ashley Smith| 9019.099999999999|
|       4249|                                 Mary Butler| 8918.849999999997|
|       5654|                                 Jerry Smith| 8904.949999999997|
|       5624|                                   Mary Mata| 8761.

# 11. What are the top 10 revenue generating products

In [50]:
orders.join(order_items, orders.order_id ==  order_items.order_item_order_id).join(products, products.product_id==order_items.order_item_product_id).\
groupBy('product_id','product_name').agg(round(sum(order_items.order_item_subtotal),2).alias('product_revenue')).\
orderBy(col('product_revenue').desc()).limit(10).show()


+----------+--------------------+---------------+
|product_id|        product_name|product_revenue|
+----------+--------------------+---------------+
|      1004|Field & Stream Sp...|      6929653.5|
|       365|Perfect Fitness P...|     4421143.02|
|       957|Diamondback Women...|     4118425.42|
|       191|Nike Men's Free 5...|      3667633.2|
|       502|Nike Men's Dri-FI...|      3147800.0|
|      1073|Pelican Sunstream...|      3099845.0|
|       403|Nike Men's CJ Eli...|     2891757.54|
|      1014|O'Brien Men's Neo...|     2888993.94|
|       627|Under Armour Girl...|     1269082.65|
|       565|adidas Youth Germ...|        67830.0|
+----------+--------------------+---------------+



# 12. Top 5 revenue generating deparments

In [52]:
order_items.join(products, products.product_id ==  order_items.order_item_product_id).\
join(categories, categories.category_id == products.product_category_id).\
join(departments, departments.department_id ==  categories.category_department_id).\
groupBy('department_id','department_name').agg(round(sum(order_items.order_item_subtotal),2).alias('dept_revenue')).\
orderBy(col('dept_revenue').desc()).limit(5).show()


+-------------+---------------+-------------+
|department_id|department_name| dept_revenue|
+-------------+---------------+-------------+
|            7|       Fan Shop|1.710776588E7|
|            4|        Apparel|    7323700.2|
|            5|           Golf|   4609028.22|
|            3|       Footwear|   4006498.77|
|            6|       Outdoors|    995582.72|
+-------------+---------------+-------------+



# 13. Top 5 revenue generating cities (from address of Customers)

In [54]:
orders.join(order_items, orders.order_id ==  order_items.order_item_order_id).\
join(customers, customers.customer_id ==  orders.order_customer_id).\
groupBy(customers.customer_city).agg(round(sum(order_items.order_item_subtotal),2).alias('city_revenue')).\
orderBy(col('city_revenue').desc()).limit(5).show()


+-------------+-------------+
|customer_city| city_revenue|
+-------------+-------------+
|       Caguas|1.269771807E7|
|      Chicago|    746869.57|
|  Los Angeles|    645083.23|
|     Brooklyn|    643507.92|
|     New York|    338937.57|
+-------------+-------------+

